In [0]:
from pyspark.sql import functions as F
df=spark.table("default.USDINR_DXY_")
initial_value=df.orderBy('Date').limit(1).collect()[0]
UINR_in=initial_value['USDINRClose']
DXY_in=initial_value['DXYClose']
#UINR=UINR.select("*",F.round((F.col('USDINRClose')-F.lit(initial_value))*100/F.lit(initial_value),2).alias('pct_change_12M'))
df = df.withColumn(
    "pct_change_USDINRC_12M",
    F.round(((F.col("USDINRClose") - F.lit(UINR_in)) * 100 / F.lit(UINR_in)), 2)
).withColumn(
    "pct_change_DXY_12M",
    F.round(((F.col("DXYClose") - F.lit(DXY_in)) * 100 / F.lit(DXY_in)), 2)
)
df.write.option("mergeSchema","true").mode("overwrite").format("delta").saveAsTable("USDINR_DXY_Pct")
df.show(5)            

In [0]:
%sql
select * from USDINR_DXY_Pct;

In [0]:
%sql
--Going with Partitioning of Table for better performance and Insert.
CREATE OR REPLACE VIEW USDINR_DXY_TEMP 
as 
select *,year(Date) AS year, month(Date) as month 
from USDINR_DXY_Pct;

In [0]:
%sql
--Creating or replaced partition table
CREATE OR REPLACE TABLE USDINR_DXY_partitioned
USING delta
PARTITIONED BY (year, month)
AS
SELECT * FROM USDINR_DXY_TEMP;

In [0]:

%sql
--Dropping the staging table
drop table if exists USDINR_DXY_;

--Dropping the other intermediate pct table
drop table if exists USDINR_DXY_Pct;

--Dropping the temp table
drop view if exists USDINR_DXY_TEMP;

In [0]:
%sql
--Checking top 100 data
select * from USDINR_DXY_partitioned
order by Date desc
limit 100;